In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q kaggle

In [3]:
!kaggle datasets download -d sudalairajkumar/telugu-nlp

Dataset URL: https://www.kaggle.com/datasets/sudalairajkumar/telugu-nlp
License(s): copyright-authors
telugu-nlp.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
 !unzip -q telugu-nlp.zip  -d train 

replace train/telugu_books/telugu_books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [5]:
df1 = pd.read_csv('train/telugu_news/test_telugu_news.csv')
df2 = pd.read_csv('train/telugu_news/test_telugu_news.csv')
books_df = pd.read_csv('train/telugu_books/telugu_books.csv')

KeyboardInterrupt: 

In [15]:
books_df = books_df.dropna()
books_df['text'] = books_df['text'].astype(str)
text = books_df['text'].to_list()
text = text[:4000]
train_text = '\n'.join(text)

In [16]:
with open('telugu_books_train_sub.txt', 'w', encoding='utf-8') as f:
    f.write(train_text)

In [17]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8000 entries, 0 to 8006
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   SNo     8000 non-null   int64 
 1   text    8000 non-null   object
dtypes: int64(1), object(1)
memory usage: 187.5+ KB


In [18]:
import sentencepiece as spm

# Path to the prepared text file
input_file = 'telugu_books_train_sub.txt'

options = dict(
  # input spec
  input=input_file,
  input_format="text",
  # output spec
  model_prefix="telugu_books_bpe_sub", # output filename prefix
  # algorithm spec
  # BPE alg
  model_type="bpe",
  vocab_size=5000,
  # normalization
  normalization_rule_name="identity", # ew, turn off normalization
  remove_extra_whitespaces=False,
  input_sentence_size=200000000, # max number of training sentences
  max_sentence_length=4192, # max number of bytes per sentence
  seed_sentencepiece_size=1000000,
  shuffle_input_sentence=True,
  # rare word treatment
  character_coverage=0.99995,
  byte_fallback=True,
  # merge rules
  split_digits=True,
  split_by_unicode_script=True,
  split_by_whitespace=True,
  split_by_number=True,
  max_sentencepiece_length=16,
  add_dummy_prefix=True,
  allow_whitespace_only_pieces=True,
  # special tokens
  unk_id=0, # the UNK token MUST exist
  bos_id=1, # the others are optional, set to -1 to turn off
  eos_id=2,
  pad_id=-1,
)

spm.SentencePieceTrainer.train(**options)


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: telugu_books_train_sub.txt
  input_format: text
  model_prefix: telugu_books_bpe_sub
  model_type: BPE
  vocab_size: 5000
  self_test_sample_size: 0
  character_coverage: 0.99995
  input_sentence_size: 200000000
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 1
  required_chars: 
  byte_fallback: 1
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  ena

In [19]:
# Load the SentencePiece model
sp = spm.SentencePieceProcessor(model_file='telugu_books_bpe_sub.model')

# Tokenize a Telugu sentence
sentence = 'మంచి ఉదయం'
tokens = sp.encode(sentence, out_type=str)
print(tokens)

# Detokenize back to the original sentence
original_sentence = sp.decode(tokens)
print(original_sentence)


['▁మంచి', '▁ఉదయం']
మంచి ఉదయం


In [20]:
def calculate_compression_ratio(original_text, sp_model):
    # Calculate the original size in characters
    original_size = len(original_text)
    
    # Tokenize the original text and calculate the tokenized size in tokens
    tokenized_text = sp_model.encode(original_text, out_type=str)
    tokenized_size = len(tokenized_text)
    
    # Calculate the compression ratio
    compression_ratio = original_size / tokenized_size
    return compression_ratio

In [21]:
calculate_compression_ratio(train_text,sp)

3.275637034271226